# Week 17: Cumulative Trading Strategy (Weeks 1-17)

## Integration: Foundation through Options & Deep Hedging

**New in Week 17:** Options pricing (Black-Scholes), Greeks, Delta Hedging

| Week Range | Topics Integrated |
|------------|------------------|
| 1-4 | Foundation, Statistics, Time Series, ML Basics |
| 5-8 | Portfolio Optimization, Linear/Factor Models, Trees, Volatility |
| 9-12 | Unsupervised Learning, Time Series Forecasting, Feature Engineering, Backtesting |
| 13-16 | Neural Networks, RNN/LSTM, Transformers, Reinforcement Learning |
| **17** | **Black-Scholes, Greeks, Delta Hedging, Deep Hedging** |

---

In [ ]:
# All imports from Weeks 1-17
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from scipy import stats
from scipy.stats import norm
from scipy.optimize import minimize, brentq
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')

TRADING_DAYS = 252
RISK_FREE_RATE = 0.05
np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
print("✅ Libraries loaded (Weeks 1-17)")

## Week 17: Black-Scholes Option Pricing

In [ ]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    """Black-Scholes option pricing (Week 17)."""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

def greeks(S, K, T, r, sigma):
    """Calculate option Greeks (Week 17)."""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = (-S * norm.pdf(d1) * sigma / (2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2)) / TRADING_DAYS
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    rho = K * T * np.exp(-r*T) * norm.cdf(d2) / 100
    
    return {'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}

# Example
S, K, T, r, sigma = 100, 100, 0.25, 0.05, 0.2
call_price = black_scholes(S, K, T, r, sigma, 'call')
option_greeks = greeks(S, K, T, r, sigma)

print(f"\n📊 BLACK-SCHOLES PRICING (Week 17)")
print(f"   Call Price: ${call_price:.2f}")
print(f"   Delta: {option_greeks['delta']:.4f}")
print(f"   Gamma: {option_greeks['gamma']:.4f}")
print(f"   Theta: ${option_greeks['theta']:.4f}/day")

In [ ]:
def delta_hedge_simulation(S0, K, T, r, sigma, n_steps=252, n_paths=1000):
    """Simulate delta hedging performance (Week 17)."""
    dt = T / n_steps
    
    # Simulate stock paths (GBM)
    Z = np.random.standard_normal((n_paths, n_steps))
    S = np.zeros((n_paths, n_steps + 1))
    S[:, 0] = S0
    
    for t in range(n_steps):
        S[:, t+1] = S[:, t] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z[:, t])
    
    # Delta hedging P&L
    hedging_pnl = []
    for path in range(min(n_paths, 100)):  # Limit for speed
        portfolio_value = black_scholes(S0, K, T, r, sigma, 'call')  # Sell call
        shares = greeks(S0, K, T, r, sigma)['delta']
        cash = portfolio_value - shares * S0
        
        for t in range(1, n_steps + 1):
            time_to_expiry = T - t * dt
            if time_to_expiry > 0:
                new_delta = greeks(S[path, t], K, time_to_expiry, r, sigma)['delta']
                trade = new_delta - shares
                cash = cash * np.exp(r * dt) - trade * S[path, t]
                shares = new_delta
        
        # Final P&L
        option_payoff = max(S[path, -1] - K, 0)
        final_pnl = shares * S[path, -1] + cash - option_payoff
        hedging_pnl.append(final_pnl)
    
    return np.array(hedging_pnl)

# Run simulation
hedging_pnl = delta_hedge_simulation(100, 100, 0.25, 0.05, 0.2)
print(f"\n🎯 DELTA HEDGING SIMULATION (Week 17)")
print(f"   Mean P&L: ${hedging_pnl.mean():.2f}")
print(f"   Std P&L: ${hedging_pnl.std():.2f}")
print(f"   Hedging Error: {hedging_pnl.std()/black_scholes(100,100,0.25,0.05,0.2)*100:.2f}%")

## Integrated Strategy: Stocks + Options Hedging

In [ ]:
# Download data
ticker = 'SPY'
data = yf.download(ticker, start='2020-01-01', end='2024-01-01', progress=False, auto_adjust=True)
prices = data['Close']
returns = prices.pct_change().dropna()

# Calculate realized volatility (Week 7)
realized_vol = returns.rolling(20).std() * np.sqrt(TRADING_DAYS)

# Regime detection (Week 9)
vol_regime = pd.cut(realized_vol, bins=[0, 0.15, 0.25, 1], labels=['Low', 'Medium', 'High'])

# Options hedging strategy based on regime
def calculate_hedge_ratio(current_vol, base_delta=1.0):
    """Adjust hedge ratio based on volatility regime."""
    if current_vol < 0.15:
        return base_delta * 1.0  # Full exposure in low vol
    elif current_vol < 0.25:
        return base_delta * 0.8  # Reduce exposure
    else:
        return base_delta * 0.5  # Significant hedge in high vol

# Simulate hedged portfolio
hedged_returns = []
for i in range(20, len(returns)):
    vol = realized_vol.iloc[i]
    hedge_ratio = calculate_hedge_ratio(vol)
    hedged_ret = returns.iloc[i] * hedge_ratio
    hedged_returns.append(hedged_ret)

hedged_returns = pd.Series(hedged_returns, index=returns.index[20:])

# Performance comparison
print("\n📈 HEDGED PORTFOLIO PERFORMANCE")
print("="*50)
for name, rets in [('Unhedged', returns[20:]), ('Vol-Hedged', hedged_returns)]:
    ann_ret = rets.mean() * TRADING_DAYS
    ann_vol = rets.std() * np.sqrt(TRADING_DAYS)
    sharpe = (ann_ret - RISK_FREE_RATE) / ann_vol
    print(f"   {name}: Return={ann_ret:.2%}, Vol={ann_vol:.2%}, Sharpe={sharpe:.2f}")

## Summary: Weeks 1-17 Integration

| Week | Concept | Implementation |
|------|---------|---------------|
| 1-2 | Data & Statistics | Price data, returns, volatility |
| 7 | Volatility Models | Realized volatility calculation |
| 9 | Regime Detection | Vol-based regime classification |
| **17** | **Options Pricing** | **Black-Scholes, Greeks, Delta Hedging** |

✅ Complete integration of options hedging with ML-based regime detection!